In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader.av.forex import AVForexReader
# !pip install pyflux
# model = pf.DynReg('Amazon ~ SP500', data=final_returns)


c:\python_venvs\finance\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [6]:
n_test=10
n_lags=10
yahoo_other=["CL=F","RUB=X"]
yahoo_stocks=["^GSPC","^DJI","IMOEX.ME","000001.SS"]
from datetime import datetime
yahoo_ticks=yahoo_stocks+yahoo_other
# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = datetime.strptime('2015-01-01','%Y-%m-%d')
end_date = datetime.strptime('2020-12-31' ,'%Y-%m-%d')

# User pandas_reader.data.DataReader to load the desired data. As simple as that.
panel_df_1= data.DataReader(yahoo_ticks, 'yahoo', start_date, end_date).dropna()
# panel_df_2= data.DataReader("EFFR", 'fred', start_date, end_date).dropna()
# df_raw=pd.concat([panel_df_1, panel_df_2], axis=1, join='outer')
df_raw=panel_df_1

In [7]:
df_joined=df_raw.copy()
df_joined.head(10)
# df_joined.xs('Close', level='Col', axis=0)
columns_to_select=[i  for i in df_joined.columns if i[0]=="Close" or i=="EFFR"]
df_joined=df_joined[columns_to_select]
df_joined.tail(3)
df_joined.columns=[i[1] if len(i)>1 else i for i in df_joined.columns]
df_joined=df_joined.dropna()
df_joined

,^GSPC,^DJI,IMOEX.ME,000001.SS,CL=F,RUB=X
Date,,,,,,
2015-01-16,2019.420044,17511.570312,1591.430054,3376.495117,48.689999,64.680000
2015-01-20,2022.550049,17515.230469,1570.969971,3173.052002,46.389999,64.806702
2015-01-21,2032.119995,17554.279297,1618.170044,3323.611084,47.779999,64.970001
2015-01-22,2063.149902,17813.980469,1666.560059,3343.343994,46.310001,65.269997
2015-01-23,2051.820068,17672.599609,1671.800049,3351.763916,45.590000,64.305000
...,...,...,...,...,...,...
2020-03-12,2480.639893,21200.619141,2286.399902,2923.486084,32.980000,73.114998
2020-03-13,2711.020020,23185.619141,2316.379883,2887.427002,32.930000,74.898102
2020-03-16,2386.129883,20188.519531,2266.899902,2789.253906,31.730000,73.237999


In [36]:
nobs = 10
# for i in range(nobs):
#     df_joined.loc[len(df_joined)] = 0
df_train, df_test = df_joined[0:-nobs], df_joined[-nobs:]
# df_train=df_joined
# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (4, 8)
df_test.tail(10)

(1014, 6)
(10, 6)


,^GSPC,^DJI,IMOEX.ME,000001.SS,CL=F,RUB=X
Date,,,,,,
2020-03-04,3130.120117,27090.859375,2828.010010,3011.666016,47.180000,66.179604
2020-03-05,3023.939941,26121.279297,2816.699951,3071.677002,46.779999,66.192001
2020-03-06,2972.370117,25864.779297,2719.510010,3034.510986,45.900002,67.539398
2020-03-10,2882.229980,25018.160156,2498.939941,2996.761963,31.129999,74.640999
2020-03-11,2741.379883,23553.220703,2492.879883,2968.517090,34.360001,71.050201
2020-03-12,2480.639893,21200.619141,2286.399902,2923.486084,32.980000,73.114998
2020-03-13,2711.020020,23185.619141,2316.379883,2887.427002,32.930000,74.898102
2020-03-16,2386.129883,20188.519531,2266.899902,2789.253906,31.730000,73.237999
2020-03-17,2529.189941,21237.380859,2224.739990,2779.641113,26.440001,74.356300


In [43]:
import re
re.search("\w+","+RUB=awd").group(0)

'RUB'

In [61]:
df_differenced = df_train.diff().dropna()
df_differenced.columns=[re.sub("[^a-zA-Z]+", "",i) for i in df_differenced.columns]
df_differenced.columns

df_differenced_test = df_test.diff().dropna()
df_differenced_test.columns=[re.sub("[^a-zA-Z]+", "",i) for i in df_differenced_test.columns]
df_differenced_test.columns
# df_differenced.columns=[i if re.search("[A-z]",i) is not None else  for i in df_differenced.columns]

# df_differenced.columns=
formula=f'RUBX ~ {"+".join([i for i in df_differenced.columns if "RUBX" not in i])}'

In [48]:
import pyflux as pf
model = pf.DynReg(formula=formula, data=df_differenced)


In [50]:
x = model.fit()
x.summary()

Dynamic Linear Regression                                                                                 
======================================================= ==================================================
Dependent Variable: RUBX                                Method: MLE                                       
Start Date: 2015-01-20 00:00:00                         Log Likelihood: -1819.9487                        
End Date: 2020-03-03 00:00:00                           AIC: 3653.8975                                    
Number of observations: 1013                            BIC: 3688.3422                                    
Latent Variable                          Estimate   Std Error  z        P>|z|    95% C.I.                 
======================================== ========== ========== ======== ======== =========================
Sigma^2 irregular                        0.2081                                                           
Sigma^2 1                            

Index(['^GSPC', '^DJI', 'IMOEX.ME', '000001.SS', 'CL=F', 'RUB=X'], dtype='object')

In [63]:
model.plot_predict(h=10,oos_data=df_differenced_test)
# df_differenced.columns

TypeError: _forecast_model() missing 1 required positional argument: 'h'

In [52]:
!pip install seaborn

You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
